In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from erisk.utils import get_spark

# bucket = "gs://dsgt-clef-erisk-2024"
bucket = "D:\SRC\DS@GT\CLEF eRisk"

# D:\SRC\DS@GT\CLEF eRisk
spark = get_spark()
spark

In [6]:
test = spark.read.parquet(f"{bucket}/test.parquet")
test.printSchema()

root
 |-- DOCNO: string (nullable = true)
 |-- PRE: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- POST: string (nullable = true)



In [7]:
training = spark.read.parquet(f"{bucket}/training.parquet")
training.printSchema()

root
 |-- DOCNO: string (nullable = true)
 |-- TEXT: string (nullable = true)



In [11]:
training.show(n=5, vertical=True, truncate=80)

-RECORD 0---------------------------------------------------------------------------------
 DOCNO | s_0_0_0                                                                          
 TEXT  | 1.ye katiliyorum                                                                 
-RECORD 1---------------------------------------------------------------------------------
 DOCNO | s_0_1_0                                                                          
 TEXT  | ok haklsn abi gold atar msn                                                      
-RECORD 2---------------------------------------------------------------------------------
 DOCNO | s_0_2_0                                                                          
 TEXT  | almanca yarrak gibi                                                              
-RECORD 3---------------------------------------------------------------------------------
 DOCNO | s_0_3_0                                                                          

In [16]:
import pandas as pd
import pyspark.sql.functions as F
# from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType, StringType
from sentence_transformers import SentenceTransformer

# import sbert model
model = SentenceTransformer("all-MiniLM-L6-v2")

# # create a pandas udf that will encode the text and return an array of doubles
@F.pandas_udf(returnType=ArrayType(DoubleType()))
def encode(x: pd.Series) -> pd.Series:
    return pd.Series(model.encode(x).tolist())


In [17]:

df_transformed = training.withColumn("embedding", encode(training["TEXT"]))
df_transformed.show(n=5, vertical=True, truncate=80)
# return df_transformed

-RECORD 0-------------------------------------------------------------------------------------
 DOCNO     | s_0_0_0                                                                          
 TEXT      | 1.ye katiliyorum                                                                 
 embedding | [0.018069807440042496, 0.0044670202769339085, 0.020407287403941154, -4.309581... 
-RECORD 1-------------------------------------------------------------------------------------
 DOCNO     | s_0_1_0                                                                          
 TEXT      | ok haklsn abi gold atar msn                                                      
 embedding | [-0.1302819401025772, 0.0033819573000073433, -0.027139434590935707, 0.0828814... 
-RECORD 2-------------------------------------------------------------------------------------
 DOCNO     | s_0_2_0                                                                          
 TEXT      | almanca yarrak gibi                  